In [34]:
# Import chunk of things
import xarray as xr
import matplotlib.dates as mdates
import numpy as np
import matplotlib
from matplotlib import rcParams
import datetime
import pandas as pd
import cftime
import pylab as plt
import netCDF4 as nc
from numpy import ma
import dask as dd
from matplotlib.colors import LinearSegmentedColormap,ListedColormap
from matplotlib import cm

## Map Plotting imports

from matplotlib.colors import LinearSegmentedColormap,ListedColormap
from matplotlib import cm
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point

#stash codes
stash_ch4 = 'mass_fraction_of_methane_in_air'
stash_co = 'mass_fraction_of_carbon_monoxide_in_air'
stash_ch4_oh = 'm01s50i041'
stash_oh = 'mass_fraction_of_hydroxyl_radical_in_air'
stash_o3 = 'mass_fraction_of_ozone_in_air'
stash_trop = 'm01s50i062'
stash_mass = 'm01s50i063'
stash_temp = 'surface_temperature'
stash_nox = 'm01s50i081'

#define functions
def molec_cm3(conc_kg_kg, mass, vol, mr):
    molec_box = conc_kg_kg*mass/mr*1000*n_a #molecules per box
    cm3 = molec_box/(vol*1e6) #molec per cm3
    return cm3

def moving_average(x, months):
    return np.convolve(x, np.ones(months), 'valid') / months

#define constants
n_a = 6.02e23

In [42]:
# do lightning nox calculations

area = nc.Dataset(f'/gws/nopw/j04/gcp_ch4/u-bl949/areacella_fx_UKESM1-0-LL_piControl_r1i1p1f2_gn.nc')
box_area = area.variables['areacella'][:]

total_area = np.sum(box_area)
area_scaled = box_area/total_area
lat_area_scaled = area_scaled[:,0]#1D array of latitude scaling values

## Fetch data
nox_data_949 = xr.open_dataset(f'/gws/nopw/j04/gcp_ch4/u-bl949/ubl949_l_nox.nc',chunks={'time':24})
nox_949 = nox_data_949.variables[stash_nox][:]

nlat = nox_data_949.latitude[:]
nlon = nox_data_949.longitude[:]
ntime = nox_data_949.time[:]

lnox = np.average(nox_949,axis=(1,3))
pnox = np.average(lnox,weights=lat_area_scaled,axis=(1))

# Calculate long term means then the anomaly

mean_3yr = moving_average(pnox,3)
mean_50yr = moving_average(pnox,50)

anomaly = mean_3yr[47:] - mean_50yr

dtime = nc.date2num(ntime,'days since 2000-01-01 00:00:00',calendar='360_day',has_year_zero=True)

In [43]:
mean_50yr.shape

(330,)